In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.metrics.pairwise import cosine_similarity
import os
import json
import warnings
warnings.filterwarnings('ignore')

# Import the modified algorithms
from modified_bee_colony import BeeClustering
from modified_ant_colony import AntColonyClustering, scale_data
from modified_fireworks import EnhancedFireworksClustering

In [2]:
class AlgorithmComparison:
    def __init__(self, data_path, cluster_counts=[5], results_dir="comparison_results"):
        """
        Framework for comparing ABC, ACO, and EFWA clustering algorithms with multiple cluster counts
        
        Parameters:
        -----------
        data_path : str
            Path to the dataset
        cluster_counts : list
            List of cluster counts to test [default: [5, 10, 15]]
        results_dir : str
            Directory to save results
        """
        self.data_path = data_path
        self.results_dir = results_dir
        self.cluster_counts = cluster_counts
        
        # Create results directory if it doesn't exist
        os.makedirs(results_dir, exist_ok=True)
        
        # Load and preprocess data
        print(f"Loading data from {data_path}")
        self.data_unscaled = np.load(data_path)
        self.data = scale_data(self.data_unscaled, feature_range=(0.1, 1.0))
        print(f"Data loaded. Shape: {self.data.shape}")
        
        # Parameters for all algorithms
        self.max_iterations = 500  # Maximum iterations
        
        # Results storage
        self.results = {}  # Dictionary to store results for each cluster count
        
    def run_comparison(self, save_results=True, extract_tweets=False, tweets_df_path=None, embeddings_path=None):
        """
        Run all algorithms (ABC, ACO, EFWA) with different cluster counts and collect results
        
        Parameters:
        -----------
        save_results : bool
            Whether to save the results
        """
        # Create a directory for this run
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        run_dir = f"{self.results_dir}/run_{timestamp}"
        os.makedirs(run_dir, exist_ok=True)
        
        # Main results table
        all_results = []
        
        # Run for each cluster count
        for n_clusters in self.cluster_counts:
            print(f"\n\n========== RUNNING COMPARISON WITH {n_clusters} CLUSTERS ==========")
            
            # Create directory for this cluster count
            cluster_dir = f"{run_dir}/clusters_{n_clusters}"
            os.makedirs(cluster_dir, exist_ok=True)
            
            # Run all algorithms
            abc_result = self._run_single_abc(n_clusters, cluster_dir)
            aco_result = self._run_single_aco(n_clusters, cluster_dir)
            efwa_result = self._run_single_efwa(n_clusters, cluster_dir)
            
            
            # Store results for this cluster count
            self.results[n_clusters] = {
                'abc': abc_result,
                'aco': aco_result,
                'efwa': efwa_result
            }
            
            # Print comparison summary
            # self._print_comparison_summary(n_clusters)
            
            # Run evaluation-based statistical analysis
            if abc_result and aco_result and efwa_result:
                stat_results = self._analyze_evaluation_based_statistics(n_clusters, cluster_dir)
                
                # Save visualizations
                if save_results:
                    self._visualize_results(n_clusters, cluster_dir)
                
                # Prepare result row
                result_row = {
                    'n_clusters': n_clusters,
                    'abc_fitness': abc_result['fitness'],
                    'aco_fitness': aco_result['fitness'],
                    'efwa_fitness': efwa_result['fitness'],
                    'abc_silhouette': abc_result['silhouette'],
                    'aco_silhouette': aco_result['silhouette'],
                    # 'efwa_silhouette': efwa_result['silhouette'],
                    'abc_time': abc_result['total_time'],
                    'aco_time': aco_result['total_time'],
                    'efwa_time': efwa_result['total_time'],
                    'abc_evaluations': abc_result['evaluations'],
                    'aco_evaluations': aco_result['evaluations'],
                    'efwa_evaluations': efwa_result['evaluations'],
                    'abc_aco_fitness_diff_pct': ((abc_result['fitness'] - aco_result['fitness']) / 
                                        max(abc_result['fitness'], aco_result['fitness'])) * 100,
                    'abc_efwa_fitness_diff_pct': ((abc_result['fitness'] - efwa_result['fitness']) / 
                                        max(abc_result['fitness'], efwa_result['fitness'])) * 100,
                    'aco_efwa_fitness_diff_pct': ((aco_result['fitness'] - efwa_result['fitness']) / 
                                        max(aco_result['fitness'], efwa_result['fitness'])) * 100,
                    'abc_aco_t_stat': stat_results['abc_aco_t_test']['t_statistic'],
                    'abc_aco_p_value': stat_results['abc_aco_t_test']['p_value'],
                    'abc_efwa_t_stat': stat_results['abc_efwa_t_test']['t_statistic'],
                    'abc_efwa_p_value': stat_results['abc_efwa_t_test']['p_value'],
                    'aco_efwa_t_stat': stat_results['aco_efwa_t_test']['t_statistic'],
                    'aco_efwa_p_value': stat_results['aco_efwa_t_test']['p_value']
                }
                
                all_results.append(result_row)
        
        # Save overall results
        if all_results and save_results:
            results_df = pd.DataFrame(all_results)
            results_df.to_csv(f"{run_dir}/overall_results.csv", index=False)
            self._visualize_overall_results(results_df, save_path=f"{run_dir}/overall_results.png")
            print(f"\nOverall results saved to {run_dir}/overall_results.csv")

        if extract_tweets and tweets_df_path and embeddings_path:
            self.extract_top_tweets(tweets_df_path, embeddings_path, run_dir)
        
        return self

    def _run_single_efwa(self, n_clusters, result_dir):
        """Run a single instance of EFWA algorithm"""
        print("\n======= RUNNING ENHANCED FIREWORKS ALGORITHM =======")
        try:
            # Start timing
            start_time = time.time()
            
            # Create EFWA instance
            efwa = EnhancedFireworksClustering(
                n_clusters=n_clusters,
                max_iter=self.max_iterations,
                n_sparks=n_clusters*2,
                n_guassian=5,
                explosion_amp=0.5,
                a_min=0.05,
                a_max=0.8,
                gaussian_amp=1.0,
                run_id=0,
                n_jobs=1
            )
            
            # Run clustering
            best_centroids, best_fitness, perf_metrics = efwa.optimize(self.data)
            
            # End timing
            end_time = time.time()
            total_time = end_time - start_time
            
            # Get final assignments
            final_assignments = efwa.assign_clusters(self.data, best_centroids)
            
            # Get results
            result = {
                'assignments': final_assignments,
                'centroids': best_centroids,
                'fitness': best_fitness,
                # 'silhouette': perf_metrics['final_silhouette'],
                'total_time': total_time,
                'evaluations': perf_metrics['fitness_evaluations'],
                'fitness_history': efwa.best_fitness_history,
                # 'silhouette_history': efwa.silhouette_history,
                'iteration_times': efwa.iteration_times,
                'fitness_evaluations_per_iteration': efwa.fitness_evaluations_per_iteration,
                'global_best_fitness_history': perf_metrics['best_fitness_history']
            }
            
            # Save visualization
            efwa.visualize_performance(show=False, save_path=f"{result_dir}/efwa_performance.png")
            
            # Save detailed results
            np.save(f"{result_dir}/efwa_assignments.npy", final_assignments)
            np.save(f"{result_dir}/efwa_centroids.npy", best_centroids)
            
            print(f"EFWA completed: Fitness = {result['fitness']:.2f}, Silhouette = result['silhouette']:.4f, Time = {total_time:.2f}s")
            
            return result
            
        except Exception as e:
            print(f"Error running EFWA with {n_clusters} clusters: {str(e)}")
            return None
    
    def _run_single_aco(self, n_clusters, result_dir):
        """Run a single instance of ACO algorithm"""
        print("\n======= RUNNING ANT COLONY OPTIMIZATION ALGORITHM =======")
        try:
            # Start timing
            start_time = time.time()
            
            # Create ACO instance
            aco = AntColonyClustering(
                n_clusters=n_clusters,
                n_ants=n_clusters*2,
                max_iterations=self.max_iterations,
                alpha=2.5,
                beta=50,
                rho=0.01,
                Q=0.1,
                learning_rate=0.15,
                exploration_phase=0.1,
                noise_gamma=2,
                run_id=0
            )
            
            # Run clustering
            final_assignments, best_centroids = aco.run(self.data)
            
            # End timing
            end_time = time.time()
            total_time = end_time - start_time
            
            # Calculate silhouette score
            silhouette = aco.compute_silhouette(self.data, final_assignments)
            
            # Get results
            result = {
                'assignments': final_assignments,
                'centroids': best_centroids,
                'fitness': aco.global_best_fitness,
                'silhouette': silhouette,
                'total_time': total_time,
                'evaluations': aco.fitness_evaluations,
                'fitness_history': aco.best_fitness_history,
                'silhouette_history': aco.silhouette_history,
                'fitness_evaluations_per_iteration': aco.fitness_evaluations_per_iteration,
                'iteration_times': aco.iteration_times,
                'global_best_fitness_history': aco.global_best_fitness_history
            }
            
            # Save visualization
            aco.visualize_performance(show=False, save_path=f"{result_dir}/aco_performance.png")
            
            # Save detailed results
            np.save(f"{result_dir}/aco_assignments.npy", final_assignments)
            np.save(f"{result_dir}/aco_centroids.npy", best_centroids)
            
            print(f"ACO completed: Fitness = {result['fitness']:.2f}, Silhouette = {result['silhouette']:.4f}, Time = {total_time:.2f}s")
            
            return result
            
        except Exception as e:
            print(f"Error running ACO with {n_clusters} clusters: {str(e)}")
            return None
    
    def _run_single_abc(self, n_clusters, result_dir):
        """Run a single instance of ABC algorithm"""
        print("\n======= RUNNING ARTIFICIAL BEE COLONY ALGORITHM =======")
        try:
            # Start timing
            start_time = time.time()
            
            # Create ABC instance
            abc = BeeClustering(
                self.data,
                n_clusters=n_clusters,
                num_bees=n_clusters*2,
                max_iter=self.max_iterations,
                scout_limit=3,
                n_jobs=-1,
                emp_per=0.5,
                onlook_per=0.2,
                run_id=0
            )
            
            # Run clustering
            best_centroids, best_fitness, perf_metrics = abc.optimize()
            
            # End timing
            end_time = time.time()
            total_time = end_time - start_time
            
            # Get final assignments
            final_assignments = abc.predict(best_centroids)
            
            # Get results
            result = {
                'assignments': final_assignments,
                'centroids': best_centroids,
                'fitness': best_fitness,
                'silhouette': perf_metrics['final_silhouette'],
                'total_time': total_time,
                'evaluations': perf_metrics['fitness_evaluations'],
                'fitness_history': abc.best_fitness_history,
                'silhouette_history': abc.silhouette_history,
                'iteration_times': abc.iteration_times,
                'fitness_evaluations_per_iteration': abc.fitness_evaluations_per_iteration,
                'best_fitness_history': perf_metrics['best_fitness_history']
            }
            
            # Save visualization
            abc.visualize_performance(show=False, save_path=f"{result_dir}/abc_performance.png")
            
            # Save detailed results
            np.save(f"{result_dir}/abc_assignments.npy", final_assignments)
            np.save(f"{result_dir}/abc_centroids.npy", best_centroids)
            
            print(f"ABC completed: Fitness = {result['fitness']:.2f}, Silhouette = {result['silhouette']:.4f}, Time = {total_time:.2f}s")
            
            return result
            
        except Exception as e:
            print(f"Error running ABC with {n_clusters} clusters: {str(e)}")
            return None
    
    def _print_comparison_summary(self, n_clusters):
        """Print a summary of the comparison results for a specific cluster count"""
        if n_clusters not in self.results:
            print(f"No results for {n_clusters} clusters. Run the comparison first.")
            return
        
        results = self.results[n_clusters]
        abc_result = results.get('abc')
        aco_result = results.get('aco')
        efwa_result = results.get('efwa')
        
        if not (abc_result and aco_result and efwa_result):
            print(f"Missing results for one or more algorithms for {n_clusters} clusters.")
            return
        
        print(f"\n======= COMPARISON SUMMARY FOR {n_clusters} CLUSTERS =======")
        
        # Create metrics list
        metrics = ['Fitness', 'Silhouette', 'Time (s)', 'Evaluations']
        
        # Create data dictionary for the summary DataFrame
        summary_data = {
            'Metric': metrics,
            'ABC': [abc_result['fitness'], abc_result['total_time'], abc_result['evaluations']],
            'ACO': [aco_result['fitness'], aco_result['total_time'], aco_result['evaluations']],
            'EFWA': [efwa_result['fitness'],efwa_result['total_time'], efwa_result['evaluations']]
        }
        
        # Calculate differences between algorithms
        summary_data['ABC vs ACO (%)'] = [
            ((abc_result['fitness'] - aco_result['fitness']) / 
            max(abc_result['fitness'], aco_result['fitness'])) * 100,
            ((abc_result['total_time'] - aco_result['total_time']) / 
            max(abc_result['total_time'], aco_result['total_time'])) * 100,
            ((abc_result['evaluations'] - aco_result['evaluations']) / 
            max(abc_result['evaluations'], aco_result['evaluations'])) * 100
        ]
        
        summary_data['ABC vs EFWA (%)'] = [
            ((abc_result['fitness'] - efwa_result['fitness']) / 
            max(abc_result['fitness'], efwa_result['fitness'])) * 100,
            ((abc_result['total_time'] - efwa_result['total_time']) / 
            max(abc_result['total_time'], efwa_result['total_time'])) * 100,
            ((abc_result['evaluations'] - efwa_result['evaluations']) / 
            max(abc_result['evaluations'], efwa_result['evaluations'])) * 100
        ]
        
        summary_data['ACO vs EFWA (%)'] = [
            ((aco_result['fitness'] - efwa_result['fitness']) / 
            max(aco_result['fitness'], efwa_result['fitness'])) * 100,
            ((aco_result['total_time'] - efwa_result['total_time']) / 
            max(aco_result['total_time'], efwa_result['total_time'])) * 100,
            ((aco_result['evaluations'] - efwa_result['evaluations']) / 
            max(aco_result['evaluations'], efwa_result['evaluations'])) * 100
        ]
        
        # Create DataFrame
        summary = pd.DataFrame(summary_data)
        
        # Set display options for better formatting
        pd.set_option('display.float_format', '{:.2f}'.format)
        
        print(summary)
        
        # Print comparison notes
        print("\nComparison Notes:")
        
        # ABC vs ACO
        abc_aco_fitness_diff = summary_data['ABC vs ACO (%)'][0]
        if abc_aco_fitness_diff > 0:
            print(f"- ABC achieved higher fitness score by {abs(abc_aco_fitness_diff):.2f}% compared to ACO")
        else:
            print(f"- ACO achieved higher fitness score by {abs(abc_aco_fitness_diff):.2f}% compared to ABC")
        
        # ABC vs EFWA
        abc_efwa_fitness_diff = summary_data['ABC vs EFWA (%)'][0]
        if abc_efwa_fitness_diff > 0:
            print(f"- ABC achieved higher fitness score by {abs(abc_efwa_fitness_diff):.2f}% compared to EFWA")
        else:
            print(f"- EFWA achieved higher fitness score by {abs(abc_efwa_fitness_diff):.2f}% compared to ABC")
        
        # ACO vs EFWA
        aco_efwa_fitness_diff = summary_data['ACO vs EFWA (%)'][0]
        if aco_efwa_fitness_diff > 0:
            print(f"- ACO achieved higher fitness score by {abs(aco_efwa_fitness_diff):.2f}% compared to EFWA")
        else:
            print(f"- EFWA achieved higher fitness score by {abs(aco_efwa_fitness_diff):.2f}% compared to ACO")
        
        # Time comparisons
        print("\nExecution Time Comparison:")
        abc_aco_time_diff = summary_data['ABC vs ACO (%)'][2]
        abc_efwa_time_diff = summary_data['ABC vs EFWA (%)'][2]
        aco_efwa_time_diff = summary_data['ACO vs EFWA (%)'][2]
        
        algorithms = ['ABC', 'ACO', 'EFWA']
        times = [abc_result['total_time'], aco_result['total_time'], efwa_result['total_time']]
        fastest_idx = times.index(min(times))
        print(f"- {algorithms[fastest_idx]} was the fastest algorithm")
        
        if fastest_idx == 0:  # ABC fastest
            print(f"  - Faster than ACO by {abs(abc_aco_time_diff):.2f}%")
            print(f"  - Faster than EFWA by {abs(abc_efwa_time_diff):.2f}%")
        elif fastest_idx == 1:  # ACO fastest
            print(f"  - Faster than ABC by {abs(abc_aco_time_diff):.2f}%")
            print(f"  - Faster than EFWA by {abs(aco_efwa_time_diff):.2f}%")
        else:  # EFWA fastest
            print(f"  - Faster than ABC by {abs(abc_efwa_time_diff):.2f}%")
            print(f"  - Faster than ACO by {abs(aco_efwa_time_diff):.2f}%")
        
        
    def _analyze_evaluation_based_statistics(self, n_clusters, result_dir):
        """
        Analyze algorithm performance based on number of fitness evaluations
        """
        if n_clusters not in self.results:
            print(f"No results for {n_clusters} clusters. Run the comparison first.")
            return {}
        
        results = self.results[n_clusters]
        abc_result = results.get('abc')
        aco_result = results.get('aco')
        efwa_result = results.get('efwa')
        
        if not (abc_result and aco_result and efwa_result):
            print(f"Missing results for one or more algorithms for {n_clusters} clusters.")
            return {}
        
        # Get fitness histories
        abc_fitness = np.array(abc_result['best_fitness_history'])
        aco_fitness = np.array(aco_result['global_best_fitness_history'])
        efwa_fitness = np.array(efwa_result['global_best_fitness_history'])
        
        # Get actual fitness evaluations per iteration
        if ('fitness_evaluations_per_iteration' in abc_result and 
            'fitness_evaluations_per_iteration' in aco_result and 
            'fitness_evaluations_per_iteration' in efwa_result):
            
            abc_evals_per_iter = np.array(abc_result['fitness_evaluations_per_iteration'])
            aco_evals_per_iter = np.array(aco_result['fitness_evaluations_per_iteration'])
            efwa_evals_per_iter = np.array(efwa_result['fitness_evaluations_per_iteration'])
            
            # Check if lengths match
            if len(abc_evals_per_iter) != len(abc_fitness):
                print(f"Warning: ABC evaluations per iteration ({len(abc_evals_per_iter)}) doesn't match fitness history length ({len(abc_fitness)})")
                # Use only available data
                min_len = min(len(abc_fitness), len(abc_evals_per_iter))
                abc_fitness = abc_fitness[:min_len]
                abc_evals_per_iter = abc_evals_per_iter[:min_len]
                
            if len(aco_evals_per_iter) != len(aco_fitness):
                print(f"Warning: ACO evaluations per iteration ({len(aco_evals_per_iter)}) doesn't match fitness history length ({len(aco_fitness)})")
                # Use only available data
                min_len = min(len(aco_fitness), len(aco_evals_per_iter))
                aco_fitness = aco_fitness[:min_len]
                aco_evals_per_iter = aco_evals_per_iter[:min_len]
                
            if len(efwa_evals_per_iter) != len(efwa_fitness):
                print(f"Warning: EFWA evaluations per iteration ({len(efwa_evals_per_iter)}) doesn't match fitness history length ({len(efwa_fitness)})")
                # Use only available data
                min_len = min(len(efwa_fitness), len(efwa_evals_per_iter))
                efwa_fitness = efwa_fitness[:min_len]
                efwa_evals_per_iter = efwa_evals_per_iter[:min_len]
        else:
            # Fallback to estimates if not available
            print("Warning: Using estimated evaluation counts. Modify algorithms to track actual counts.")
            num_bees = n_clusters * 2  # From parameters
            n_ants = n_clusters * 2    # From parameters
            n_fireworks = n_clusters * 2  # From parameters
            abc_evals_per_iter = np.full(len(abc_fitness), 2 * num_bees)
            aco_evals_per_iter = np.full(len(aco_fitness), n_ants)
            efwa_evals_per_iter = np.full(len(efwa_fitness), n_fireworks)
        
        # Create arrays of cumulative evaluations
        abc_cum_evals = np.cumsum(abc_evals_per_iter)
        aco_cum_evals = np.cumsum(aco_evals_per_iter)
        efwa_cum_evals = np.cumsum(efwa_evals_per_iter)
        
        # Create interpolation functions for evaluation-based comparison
        from scipy.interpolate import interp1d
        
        # Ensure at least 2 points for interpolation
        if len(abc_cum_evals) <= 1 or len(aco_cum_evals) <= 1 or len(efwa_cum_evals) <= 1:
            print("Not enough evaluation points for interpolation")
            return {'abc_aco_t_test': {'t_statistic': 0, 'p_value': 1.0, 'significant': False},
                    'abc_efwa_t_test': {'t_statistic': 0, 'p_value': 1.0, 'significant': False},
                    'aco_efwa_t_test': {'t_statistic': 0, 'p_value': 1.0, 'significant': False}}
        
        # Add a small epsilon to avoid duplicates if present
        for i in range(1, len(abc_cum_evals)):
            if abc_cum_evals[i] <= abc_cum_evals[i-1]:
                abc_cum_evals[i] = abc_cum_evals[i-1] + 0.001
                
        for i in range(1, len(aco_cum_evals)):
            if aco_cum_evals[i] <= aco_cum_evals[i-1]:
                aco_cum_evals[i] = aco_cum_evals[i-1] + 0.001
                
        for i in range(1, len(efwa_cum_evals)):
            if efwa_cum_evals[i] <= efwa_cum_evals[i-1]:
                efwa_cum_evals[i] = efwa_cum_evals[i-1] + 0.001
        
        # Create interpolation functions
        abc_interp = interp1d(abc_cum_evals, abc_fitness, bounds_error=False, fill_value=(abc_fitness[0], abc_fitness[-1]))
        aco_interp = interp1d(aco_cum_evals, aco_fitness, bounds_error=False, fill_value=(aco_fitness[0], aco_fitness[-1]))
        efwa_interp = interp1d(efwa_cum_evals, efwa_fitness, bounds_error=False, fill_value=(efwa_fitness[0], efwa_fitness[-1]))
        
        # Create common evaluation points for comparison
        max_evals = min(abc_cum_evals[-1], aco_cum_evals[-1], efwa_cum_evals[-1])
        min_evals = max(abc_cum_evals[0], aco_cum_evals[0], efwa_cum_evals[0])
        
        # Create 50 equally spaced evaluation points for comparison
        common_evals = np.linspace(min_evals, max_evals, 50)
        
        # Interpolate fitness values at common evaluation points
        abc_interp_fitness = abc_interp(common_evals)
        aco_interp_fitness = aco_interp(common_evals)
        efwa_interp_fitness = efwa_interp(common_evals)
        
        # Calculate fitness differences for pairwise comparisons
        abc_aco_fitness_diffs = abc_interp_fitness - aco_interp_fitness
        abc_efwa_fitness_diffs = abc_interp_fitness - efwa_interp_fitness
        aco_efwa_fitness_diffs = aco_interp_fitness - efwa_interp_fitness
        
        # Paired t-tests on evaluation-aligned performance
        abc_aco_t_stat, abc_aco_p_value = stats.ttest_rel(abc_interp_fitness, aco_interp_fitness)
        abc_efwa_t_stat, abc_efwa_p_value = stats.ttest_rel(abc_interp_fitness, efwa_interp_fitness)
        aco_efwa_t_stat, aco_efwa_p_value = stats.ttest_rel(aco_interp_fitness, efwa_interp_fitness)
        
        # Calculate means and standard deviations of differences
        abc_aco_mean_diff = np.mean(abc_aco_fitness_diffs)
        abc_aco_std_diff = np.std(abc_aco_fitness_diffs)
        
        abc_efwa_mean_diff = np.mean(abc_efwa_fitness_diffs)
        abc_efwa_std_diff = np.std(abc_efwa_fitness_diffs)
        
        aco_efwa_mean_diff = np.mean(aco_efwa_fitness_diffs)
        aco_efwa_std_diff = np.std(aco_efwa_fitness_diffs)
        
        # Calculate 95% confidence intervals
        n = len(common_evals)
        abc_aco_confidence_interval = stats.t.interval(0.95, n-1, loc=abc_aco_mean_diff, scale=abc_aco_std_diff/np.sqrt(n))
        abc_efwa_confidence_interval = stats.t.interval(0.95, n-1, loc=abc_efwa_mean_diff, scale=abc_efwa_std_diff/np.sqrt(n))
        aco_efwa_confidence_interval = stats.t.interval(0.95, n-1, loc=aco_efwa_mean_diff, scale=aco_efwa_std_diff/np.sqrt(n))
        
        # Create visualization
        plt.figure(figsize=(18, 12))
        
        # 1. Fitness vs Evaluations (raw)
        plt.subplot(2, 3, 1)
        plt.plot(abc_cum_evals, abc_fitness, 'b-', label='ABC')
        plt.plot(aco_cum_evals, aco_fitness, 'r-', label='ACO')
        plt.plot(efwa_cum_evals, efwa_fitness, 'g-', label='EFWA')
        plt.title(f'Fitness vs. Evaluations ({n_clusters} clusters)')
        plt.xlabel('Number of Fitness Evaluations')
        plt.ylabel('Fitness Score')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 2. Interpolated Fitness vs Evaluations
        plt.subplot(2, 3, 2)
        plt.plot(common_evals, abc_interp_fitness, 'b-', label='ABC')
        plt.plot(common_evals, aco_interp_fitness, 'r-', label='ACO')
        plt.plot(common_evals, efwa_interp_fitness, 'g-', label='EFWA')
        plt.title('Evaluation-Aligned Fitness Comparison')
        plt.xlabel('Number of Fitness Evaluations')
        plt.ylabel('Fitness Score')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 3. ABC vs ACO Fitness differences
        plt.subplot(2, 3, 3)
        plt.plot(common_evals, abc_aco_fitness_diffs, 'purple', label='ABC-ACO')
        plt.axhline(y=0, color='k', linestyle='--', alpha=0.5)
        plt.fill_between(common_evals, 
                        [abc_aco_confidence_interval[0]] * len(common_evals),
                        [abc_aco_confidence_interval[1]] * len(common_evals),
                        color='purple', alpha=0.2)
        plt.title(f'ABC vs ACO Difference\np={abc_aco_p_value:.4f}')
        plt.xlabel('Number of Fitness Evaluations')
        plt.ylabel('Fitness Difference')
        plt.grid(True, alpha=0.3)
        
        # 4. ABC vs EFWA Fitness differences
        plt.subplot(2, 3, 4)
        plt.plot(common_evals, abc_efwa_fitness_diffs, 'green', label='ABC-EFWA')
        plt.axhline(y=0, color='k', linestyle='--', alpha=0.5)
        plt.fill_between(common_evals, 
                        [abc_efwa_confidence_interval[0]] * len(common_evals),
                        [abc_efwa_confidence_interval[1]] * len(common_evals),
                        color='green', alpha=0.2)
        plt.title(f'ABC vs EFWA Difference\np={abc_efwa_p_value:.4f}')
        plt.xlabel('Number of Fitness Evaluations')
        plt.ylabel('Fitness Difference')
        plt.grid(True, alpha=0.3)
        
        # 5. ACO vs EFWA Fitness differences
        plt.subplot(2, 3, 5)
        plt.plot(common_evals, aco_efwa_fitness_diffs, 'orange', label='ACO-EFWA')
        plt.axhline(y=0, color='k', linestyle='--', alpha=0.5)
        plt.fill_between(common_evals, 
                        [aco_efwa_confidence_interval[0]] * len(common_evals),
                        [aco_efwa_confidence_interval[1]] * len(common_evals),
                        color='orange', alpha=0.2)
        plt.title(f'ACO vs EFWA Difference\np={aco_efwa_p_value:.4f}')
        plt.xlabel('Number of Fitness Evaluations')
        plt.ylabel('Fitness Difference')
        plt.grid(True, alpha=0.3)
        
        # 6. Evaluations per iteration comparison
        plt.subplot(2, 3, 6)
        plt.bar(['ABC', 'ACO', 'EFWA'], 
               [np.mean(abc_evals_per_iter), np.mean(aco_evals_per_iter), np.mean(efwa_evals_per_iter)],
               color=['blue', 'red', 'green'])
        plt.title('Average Evaluations Per Iteration')
        plt.ylabel('Number of Evaluations')
        plt.grid(True, alpha=0.3)
        
        plt.suptitle(f'Evaluation-Based Statistical Analysis ({n_clusters} clusters)', fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.savefig(f"{result_dir}/evaluation_based_statistics.png")
        plt.close()
        
        # Calculate area under curve for each algorithm
        abc_auc = np.trapz(abc_interp_fitness, common_evals)
        aco_auc = np.trapz(aco_interp_fitness, common_evals)
        efwa_auc = np.trapz(efwa_interp_fitness, common_evals)
        
        # Calculate relative differences as percentages
        abc_aco_auc_diff_percent = ((abc_auc - aco_auc) / max(abc_auc, aco_auc)) * 100
        abc_efwa_auc_diff_percent = ((abc_auc - efwa_auc) / max(abc_auc, efwa_auc)) * 100
        aco_efwa_auc_diff_percent = ((aco_auc - efwa_auc) / max(aco_auc, efwa_auc)) * 100
        
        # Calculate fitness per evaluation efficiency
        abc_fitness_per_eval = abc_result['fitness'] / abc_result['evaluations']
        aco_fitness_per_eval = aco_result['fitness'] / aco_result['evaluations']
        efwa_fitness_per_eval = efwa_result['fitness'] / efwa_result['evaluations']
        
        # Calculate average improvement rates
        if len(abc_fitness) > 1 and len(aco_fitness) > 1 and len(efwa_fitness) > 1:
            abc_fitness_diff = np.diff(abc_fitness)
            aco_fitness_diff = np.diff(aco_fitness)
            efwa_fitness_diff = np.diff(efwa_fitness)
            
            # Make sure cum_evals are the right size for diff operation
            abc_evals_diff = np.diff(abc_cum_evals)
            aco_evals_diff = np.diff(aco_cum_evals)
            efwa_evals_diff = np.diff(efwa_cum_evals)
            
            # Avoid division by zero
            abc_evals_diff = np.maximum(abc_evals_diff, 1)
            aco_evals_diff = np.maximum(aco_evals_diff, 1)
            efwa_evals_diff = np.maximum(efwa_evals_diff, 1)
            
            abc_improvement_rate = abc_fitness_diff / abc_evals_diff
            aco_improvement_rate = aco_fitness_diff / aco_evals_diff
            efwa_improvement_rate = efwa_fitness_diff / efwa_evals_diff
            
            # Calculate evaluation efficiency visualization
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
            
            # Plot fitness per evaluation
            ax1.bar(['ABC', 'ACO', 'EFWA'], 
                   [abc_fitness_per_eval, aco_fitness_per_eval, efwa_fitness_per_eval],
                   color=['blue', 'red', 'green'])
            ax1.set_title('Fitness per Evaluation')
            ax1.set_ylabel('Score / Evaluation')
            ax1.grid(True, alpha=0.3)
            
            # Plot improvement rate
            ax2.plot(abc_cum_evals[1:], abc_improvement_rate, 'b-', label='ABC')
            ax2.plot(aco_cum_evals[1:], aco_improvement_rate, 'r-', label='ACO')
            ax2.plot(efwa_cum_evals[1:], efwa_improvement_rate, 'g-', label='EFWA')
            ax2.set_title('Improvement Rate per Evaluation')
            ax2.set_xlabel('Cumulative Evaluations')
            ax2.set_ylabel('Fitness Improvement / Evaluation')
            ax2.legend()
            ax2.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.savefig(f"{result_dir}/evaluation_efficiency.png")
            plt.close()
            
            # Get mean improvement rates
            abc_avg_improvement = np.mean(abc_improvement_rate)
            aco_avg_improvement = np.mean(aco_improvement_rate)
            efwa_avg_improvement = np.mean(efwa_improvement_rate)
        else:
            # Not enough points for improvement rates
            abc_avg_improvement = 0
            aco_avg_improvement = 0
            efwa_avg_improvement = 0
        
        # Compile statistics results
        stats_results = {
            'abc_aco_t_test': {
                't_statistic': abc_aco_t_stat,
                'p_value': abc_aco_p_value,
                'significant': abc_aco_p_value < 0.05,
                'better_algorithm': 'ABC' if abc_aco_mean_diff > 0 else 'ACO' if abc_aco_mean_diff < 0 else 'Tie'
            },
            'abc_efwa_t_test': {
                't_statistic': abc_efwa_t_stat,
                'p_value': abc_efwa_p_value,
                'significant': abc_efwa_p_value < 0.05,
                'better_algorithm': 'ABC' if abc_efwa_mean_diff > 0 else 'EFWA' if abc_efwa_mean_diff < 0 else 'Tie'
            },
            'aco_efwa_t_test': {
                't_statistic': aco_efwa_t_stat,
                'p_value': aco_efwa_p_value,
                'significant': aco_efwa_p_value < 0.05,
                'better_algorithm': 'ACO' if aco_efwa_mean_diff > 0 else 'EFWA' if aco_efwa_mean_diff < 0 else 'Tie'
            },
            'mean_differences': {
                'abc_aco': abc_aco_mean_diff,
                'abc_efwa': abc_efwa_mean_diff,
                'aco_efwa': aco_efwa_mean_diff
            },
            'confidence_intervals': {
                'abc_aco': abc_aco_confidence_interval,
                'abc_efwa': abc_efwa_confidence_interval,
                'aco_efwa': aco_efwa_confidence_interval
            },
            'area_under_curve': {
                'abc': abc_auc,
                'aco': aco_auc,
                'efwa': efwa_auc,
                'abc_aco_percent_diff': abc_aco_auc_diff_percent,
                'abc_efwa_percent_diff': abc_efwa_auc_diff_percent,
                'aco_efwa_percent_diff': aco_efwa_auc_diff_percent
            },
            'evaluation_efficiency': {
                'abc_evals_per_iter': np.mean(abc_evals_per_iter),
                'aco_evals_per_iter': np.mean(aco_evals_per_iter),
                'efwa_evals_per_iter': np.mean(efwa_evals_per_iter),
                'abc_fitness_per_eval': abc_fitness_per_eval,
                'aco_fitness_per_eval': aco_fitness_per_eval,
                'efwa_fitness_per_eval': efwa_fitness_per_eval,
                'abc_avg_improvement': abc_avg_improvement,
                'aco_avg_improvement': aco_avg_improvement,
                'efwa_avg_improvement': efwa_avg_improvement
            }
        }
        
        # Save statistics to CSV file for easier analysis
        flat_stats = {
            'n_clusters': n_clusters,
            'abc_aco_t_stat': abc_aco_t_stat,
            'abc_aco_p_value': abc_aco_p_value,
            'abc_aco_significant': abc_aco_p_value < 0.05,
            'abc_efwa_t_stat': abc_efwa_t_stat,
            'abc_efwa_p_value': abc_efwa_p_value,
            'abc_efwa_significant': abc_efwa_p_value < 0.05,
            'aco_efwa_t_stat': aco_efwa_t_stat,
            'aco_efwa_p_value': aco_efwa_p_value,
            'aco_efwa_significant': aco_efwa_p_value < 0.05,
            'abc_aco_mean_diff': abc_aco_mean_diff,
            'abc_efwa_mean_diff': abc_efwa_mean_diff,
            'aco_efwa_mean_diff': aco_efwa_mean_diff,
            'abc_auc': abc_auc,
            'aco_auc': aco_auc,
            'efwa_auc': efwa_auc,
            'abc_aco_auc_diff_pct': abc_aco_auc_diff_percent,
            'abc_efwa_auc_diff_pct': abc_efwa_auc_diff_percent,
            'aco_efwa_auc_diff_pct': aco_efwa_auc_diff_percent,
            'abc_evals_per_iter': np.mean(abc_evals_per_iter),
            'aco_evals_per_iter': np.mean(aco_evals_per_iter),
            'efwa_evals_per_iter': np.mean(efwa_evals_per_iter),
            'abc_fitness_per_eval': abc_fitness_per_eval,
            'aco_fitness_per_eval': aco_fitness_per_eval,
            'efwa_fitness_per_eval': efwa_fitness_per_eval,
            'abc_avg_improvement': abc_avg_improvement,
            'aco_avg_improvement': aco_avg_improvement,
            'efwa_avg_improvement': efwa_avg_improvement
        }
        
        # Create a DataFrame with one row
        stats_df = pd.DataFrame([flat_stats])
        
        # Save to CSV
        stats_df.to_csv(f"{result_dir}/evaluation_statistics.csv", index=False)
        
        # Print statistical summary
        print(f"\n======= EVALUATION-BASED STATISTICAL ANALYSIS FOR {n_clusters} CLUSTERS =======")
        
        # ABC vs ACO comparison
        print("\nABC vs ACO Comparison:")
        print(f"Paired t-test: t={abc_aco_t_stat:.4f}, p={abc_aco_p_value:.4f}")
        if abc_aco_p_value < 0.05:
            if abc_aco_mean_diff > 0:
                print(f"ABC performs significantly better than ACO per evaluation (p < 0.05)")
            else:
                print(f"ACO performs significantly better than ABC per evaluation (p < 0.05)")
        else:
            print("No significant difference between ABC and ACO based on evaluation-aligned performance")
        
        # ABC vs EFWA comparison
        print("\nABC vs EFWA Comparison:")
        print(f"Paired t-test: t={abc_efwa_t_stat:.4f}, p={abc_efwa_p_value:.4f}")
        if abc_efwa_p_value < 0.05:
            if abc_efwa_mean_diff > 0:
                print(f"ABC performs significantly better than EFWA per evaluation (p < 0.05)")
            else:
                print(f"EFWA performs significantly better than ABC per evaluation (p < 0.05)")
        else:
            print("No significant difference between ABC and EFWA based on evaluation-aligned performance")
        
        # ACO vs EFWA comparison
        print("\nACO vs EFWA Comparison:")
        print(f"Paired t-test: t={aco_efwa_t_stat:.4f}, p={aco_efwa_p_value:.4f}")
        if aco_efwa_p_value < 0.05:
            if aco_efwa_mean_diff > 0:
                print(f"ACO performs significantly better than EFWA per evaluation (p < 0.05)")
            else:
                print(f"EFWA performs significantly better than ACO per evaluation (p < 0.05)")
        else:
            print("No significant difference between ACO and EFWA based on evaluation-aligned performance")
        
        print(f"\nEvaluation efficiency comparison:")
        print(f"ABC average evaluations per iteration: {np.mean(abc_evals_per_iter):.2f}")
        print(f"ACO average evaluations per iteration: {np.mean(aco_evals_per_iter):.2f}")
        print(f"EFWA average evaluations per iteration: {np.mean(efwa_evals_per_iter):.2f}")
        print(f"ABC fitness/evaluation: {abc_fitness_per_eval:.6f}")
        print(f"ACO fitness/evaluation: {aco_fitness_per_eval:.6f}")
        print(f"EFWA fitness/evaluation: {efwa_fitness_per_eval:.6f}")
        
        # Determine which algorithm has the best fitness per evaluation
        best_fitness_per_eval = max(abc_fitness_per_eval, aco_fitness_per_eval, efwa_fitness_per_eval)
        if best_fitness_per_eval == abc_fitness_per_eval:
            print(f"ABC has the best fitness per evaluation efficiency")
        elif best_fitness_per_eval == aco_fitness_per_eval:
            print(f"ACO has the best fitness per evaluation efficiency")
        else:
            print(f"EFWA has the best fitness per evaluation efficiency")
        
        return stats_results
    
    def _visualize_results(self, n_clusters, result_dir):
        """Create visualizations for a specific cluster count"""
        if n_clusters not in self.results:
            print(f"No results for {n_clusters} clusters. Run the comparison first.")
            return
        
        results = self.results[n_clusters]
        abc_result = results.get('abc')
        aco_result = results.get('aco')
        efwa_result = results.get('efwa')
        
        if not (abc_result and aco_result and efwa_result):
            print(f"Missing results for one or more algorithms for {n_clusters} clusters.")
            return
        
        # Fitness history comparison
        plt.figure(figsize=(15, 12))
        
        # Fitness history
        plt.subplot(2, 2, 1)
        plt.plot(abc_result['best_fitness_history'], 'b-', label='ABC')
        plt.plot(aco_result['global_best_fitness_history'], 'r-', label='ACO')
        plt.plot(efwa_result['global_best_fitness_history'], 'g-', label='EFWA')
        plt.title(f'Fitness History Comparison ({n_clusters} clusters)')
        plt.xlabel('Iteration')
        plt.ylabel('Fitness Score')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # # Silhouette history
        # plt.subplot(2, 2, 2)
        # if ('silhouette_history' in abc_result and 
        #     'silhouette_history' in aco_result and 
        #     'silhouette_history' in efwa_result):
            
        #     if (len(abc_result['silhouette_history']) > 0 and 
        #         len(aco_result['silhouette_history']) > 0 and
        #         len(efwa_result['silhouette_history']) > 0):
                
        #         abc_silhouette_iters = [point[0] for point in abc_result['silhouette_history']]
        #         abc_silhouette_vals = [point[1] for point in abc_result['silhouette_history']]
                
        #         aco_silhouette_iters = [point[0] for point in aco_result['silhouette_history']]
        #         aco_silhouette_vals = [point[1] for point in aco_result['silhouette_history']]
                
        #         efwa_silhouette_iters = [point[0] for point in efwa_result['silhouette_history']]
        #         efwa_silhouette_vals = [point[1] for point in efwa_result['silhouette_history']]
                
        #         plt.plot(abc_silhouette_iters, abc_silhouette_vals, 'bo-', label='ABC')
        #         plt.plot(aco_silhouette_iters, aco_silhouette_vals, 'ro-', label='ACO')
        #         plt.plot(efwa_silhouette_iters, efwa_silhouette_vals, 'go-', label='EFWA')
        #         plt.title('Silhouette Score History')
        #         plt.xlabel('Iteration')
        #         plt.ylabel('Silhouette Score')
        #         plt.legend()
        #         plt.grid(True, alpha=0.3)
        
        # Time per iteration
        plt.subplot(2, 2, 3)
        plt.plot(abc_result['iteration_times'], 'b-', label='ABC')
        plt.plot(aco_result['iteration_times'], 'r-', label='ACO')
        plt.plot(efwa_result['iteration_times'], 'g-', label='EFWA')
        plt.title('Iteration Times')
        plt.xlabel('Iteration')
        plt.ylabel('Time (seconds)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Evaluations per iteration
        plt.subplot(2, 2, 4)
        plt.plot(abc_result['fitness_evaluations_per_iteration'], 'b-', label='ABC')
        plt.plot(aco_result['fitness_evaluations_per_iteration'], 'r-', label='ACO')
        plt.plot(efwa_result['fitness_evaluations_per_iteration'], 'g-', label='EFWA')
        plt.title('Evaluations Per Iteration')
        plt.xlabel('Iteration')
        plt.ylabel('Number of Evaluations')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.suptitle(f'Algorithm Comparison for {n_clusters} Clusters', fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.savefig(f"{result_dir}/comparison_plots.png")
        plt.close()
        
        # Create a second figure for cumulative metrics
        plt.figure(figsize=(15, 10))
        
        # Cumulative fitness improvement
        # Get the minimum length of both arrays
        min_length_abc_result = min(len(np.cumsum(abc_result['iteration_times'])), len(abc_result['best_fitness_history']))
        min_length_aco_result= min(len(np.cumsum(aco_result['iteration_times'])), len(aco_result['global_best_fitness_history']))
        min_length_efwa_result = min(len(np.cumsum(efwa_result['iteration_times'])), len(efwa_result['global_best_fitness_history']))
        
        plt.subplot(2, 2, 1)
        plt.plot(np.cumsum(abc_result['iteration_times'])[:min_length_abc_result], abc_result['best_fitness_history'][:min_length_abc_result], 'b-', label='ABC')
        plt.plot(np.cumsum(aco_result['iteration_times'])[:min_length_aco_result], aco_result['global_best_fitness_history'][:min_length_aco_result], 'r-', label='ACO')
        plt.plot(np.cumsum(efwa_result['iteration_times'])[:min_length_efwa_result], efwa_result['global_best_fitness_history'][:min_length_efwa_result], 'g-', label='EFWA')
        plt.title('Fitness vs. Cumulative Time')
        plt.xlabel('Cumulative Time (seconds)')
        plt.ylabel('Fitness Score')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Cumulative evaluations
        plt.subplot(2, 2, 2)
        abc_cum_evals = np.cumsum(abc_result['fitness_evaluations_per_iteration'])
        aco_cum_evals = np.cumsum(aco_result['fitness_evaluations_per_iteration'])
        efwa_cum_evals = np.cumsum(efwa_result['fitness_evaluations_per_iteration'])
        
        plt.plot(range(len(abc_cum_evals)), abc_cum_evals, 'b-', label='ABC')
        plt.plot(range(len(aco_cum_evals)), aco_cum_evals, 'r-', label='ACO')
        plt.plot(range(len(efwa_cum_evals)), efwa_cum_evals, 'g-', label='EFWA')
        plt.title('Cumulative Evaluations')
        plt.xlabel('Iteration')
        plt.ylabel('Cumulative Evaluations')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Final metrics comparison
        plt.subplot(2, 2, 3)
        metrics = ['Fitness', 'Time (s)']
        abc_values = [abc_result['fitness'], abc_result['total_time']]
        aco_values = [aco_result['fitness'], aco_result['total_time']]
        efwa_values = [efwa_result['fitness'], efwa_result['total_time']]
        
        x = np.arange(len(metrics))
        width = 0.25
        
        plt.bar(x - width, abc_values, width, label='ABC', color='blue')
        plt.bar(x, aco_values, width, label='ACO', color='red')
        plt.bar(x + width, efwa_values, width, label='EFWA', color='green')
        
        plt.ylabel('Value')
        plt.title('Final Metrics Comparison')
        plt.xticks(x, metrics)
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Evaluation efficiency
        plt.subplot(2, 2, 4)
        abc_eff = abc_result['fitness'] / abc_result['evaluations']
        aco_eff = aco_result['fitness'] / aco_result['evaluations']
        efwa_eff = efwa_result['fitness'] / efwa_result['evaluations']
        
        plt.bar(['ABC', 'ACO', 'EFWA'], [abc_eff, aco_eff, efwa_eff], color=['blue', 'red', 'green'])
        plt.title('Fitness per Evaluation')
        plt.ylabel('Fitness / Evaluation')
        plt.grid(True, alpha=0.3)
        
        plt.suptitle(f'Cumulative Metrics for {n_clusters} Clusters', fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.savefig(f"{result_dir}/cumulative_metrics.png")
        plt.close()
    
    def _visualize_overall_results(self, results_df, save_path=None):
        """Visualize overall results across different cluster counts"""
        if len(results_df) == 0:
            print("No results to visualize.")
            return
        
        plt.figure(figsize=(18, 15))
        
        # 1. Fitness comparison
        plt.subplot(3, 2, 1)
        plt.plot(results_df['n_clusters'], results_df['abc_fitness'], 'bo-', label='ABC')
        plt.plot(results_df['n_clusters'], results_df['aco_fitness'], 'ro-', label='ACO')
        plt.plot(results_df['n_clusters'], results_df['efwa_fitness'], 'go-', label='EFWA')
        plt.title('Fitness Score by Cluster Count')
        plt.xlabel('Number of Clusters')
        plt.ylabel('Fitness Score')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # # 2. Silhouette comparison
        # plt.subplot(3, 2, 2)
        # plt.plot(results_df['n_clusters'], results_df['abc_silhouette'], 'bo-', label='ABC')
        # plt.plot(results_df['n_clusters'], results_df['aco_silhouette'], 'ro-', label='ACO')
        # plt.plot(results_df['n_clusters'], results_df['efwa_silhouette'], 'go-', label='EFWA')
        # plt.title('Silhouette Score by Cluster Count')
        # plt.xlabel('Number of Clusters')
        # plt.ylabel('Silhouette Score')
        # plt.legend()
        # plt.grid(True, alpha=0.3)
        
        # 3. Execution time comparison
        plt.subplot(3, 2, 3)
        plt.plot(results_df['n_clusters'], results_df['abc_time'], 'bo-', label='ABC')
        plt.plot(results_df['n_clusters'], results_df['aco_time'], 'ro-', label='ACO')
        plt.plot(results_df['n_clusters'], results_df['efwa_time'], 'go-', label='EFWA')
        plt.title('Execution Time by Cluster Count')
        plt.xlabel('Number of Clusters')
        plt.ylabel('Time (seconds)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 4. Fitness difference percentages
        plt.subplot(3, 2, 4)
        
        # Create a grouped bar chart for fitness differences
        x = np.arange(len(results_df['n_clusters']))
        width = 0.25
        
        plt.bar(x - width, results_df['abc_aco_fitness_diff_pct'], width, label='ABC vs ACO', color='purple')
        plt.bar(x, results_df['abc_efwa_fitness_diff_pct'], width, label='ABC vs EFWA', color='green')
        plt.bar(x + width, results_df['aco_efwa_fitness_diff_pct'], width, label='ACO vs EFWA', color='orange')
        
        plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
        plt.title('Fitness Difference Percentages')
        plt.xlabel('Number of Clusters')
        plt.ylabel('Difference (%)')
        plt.xticks(x, results_df['n_clusters'])
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 5. Statistical significance (p-values)
        plt.subplot(3, 2, 5)
        plt.plot(results_df['n_clusters'], results_df['abc_aco_p_value'], 'purple', marker='o', label='ABC vs ACO')
        plt.plot(results_df['n_clusters'], results_df['abc_efwa_p_value'], 'green', marker='o', label='ABC vs EFWA')
        plt.plot(results_df['n_clusters'], results_df['aco_efwa_p_value'], 'orange', marker='o', label='ACO vs EFWA')
        plt.axhline(y=0.05, color='r', linestyle='--', label='p=0.05')
        plt.title('Statistical Significance (p-values)')
        plt.xlabel('Number of Clusters')
        plt.ylabel('p-value')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 6. Evaluation efficiency
        plt.subplot(3, 2, 6)
        
        # Create a grouped bar chart for evaluations
        x = np.arange(len(results_df['n_clusters']))
        width = 0.25
        
        plt.bar(x - width, results_df['abc_evaluations'], width, label='ABC', color='blue')
        plt.bar(x, results_df['aco_evaluations'], width, label='ACO', color='red')
        plt.bar(x + width, results_df['efwa_evaluations'], width, label='EFWA', color='green')
        
        plt.title('Total Evaluations by Cluster Count')
        plt.xlabel('Number of Clusters')
        plt.ylabel('Number of Evaluations')
        plt.xticks(x, results_df['n_clusters'])
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.suptitle('Algorithm Comparison Across Different Cluster Counts', fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        
        if save_path:
            plt.savefig(save_path)
            print(f"Overall results visualization saved to {save_path}")
        
        plt.close()
        
        # Create a second visualization for evaluation efficiency metrics
        plt.figure(figsize=(15, 10))
        
        # 1. Fitness per evaluation by cluster count
        plt.subplot(2, 2, 1)
        # Calculate fitness per evaluation for each algorithm and cluster count
        abc_fitness_per_eval = results_df['abc_fitness'] / results_df['abc_evaluations']
        aco_fitness_per_eval = results_df['aco_fitness'] / results_df['aco_evaluations']
        efwa_fitness_per_eval = results_df['efwa_fitness'] / results_df['efwa_evaluations']
        
        plt.plot(results_df['n_clusters'], abc_fitness_per_eval, 'bo-', label='ABC')
        plt.plot(results_df['n_clusters'], aco_fitness_per_eval, 'ro-', label='ACO')
        plt.plot(results_df['n_clusters'], efwa_fitness_per_eval, 'go-', label='EFWA')
        plt.title('Fitness per Evaluation')
        plt.xlabel('Number of Clusters')
        plt.ylabel('Fitness / Evaluation')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 2. T-statistics comparison
        plt.subplot(2, 2, 2)
        plt.plot(results_df['n_clusters'], results_df['abc_aco_t_stat'], 'purple', marker='o', label='ABC vs ACO')
        plt.plot(results_df['n_clusters'], results_df['abc_efwa_t_stat'], 'green', marker='o', label='ABC vs EFWA')
        plt.plot(results_df['n_clusters'], results_df['aco_efwa_t_stat'], 'orange', marker='o', label='ACO vs EFWA')
        plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
        plt.title('T-Statistics by Cluster Count')
        plt.xlabel('Number of Clusters')
        plt.ylabel('T-Statistic')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 3. Average evaluations per iteration
        plt.subplot(2, 2, 3)
        
        # Need to extract this data from the detailed results
        n_clusters_list = results_df['n_clusters'].values
        abc_avg_evals_per_iter = []
        aco_avg_evals_per_iter = []
        efwa_avg_evals_per_iter = []
        
        for n_clusters in n_clusters_list:
            results = self.results[n_clusters]
            abc_result = results.get('abc')
            aco_result = results.get('aco')
            efwa_result = results.get('efwa')
            
            if abc_result and 'fitness_evaluations_per_iteration' in abc_result:
                abc_avg_evals_per_iter.append(np.mean(abc_result['fitness_evaluations_per_iteration']))
            else:
                abc_avg_evals_per_iter.append(0)
                
            if aco_result and 'fitness_evaluations_per_iteration' in aco_result:
                aco_avg_evals_per_iter.append(np.mean(aco_result['fitness_evaluations_per_iteration']))
            else:
                aco_avg_evals_per_iter.append(0)
                
            if efwa_result and 'fitness_evaluations_per_iteration' in efwa_result:
                efwa_avg_evals_per_iter.append(np.mean(efwa_result['fitness_evaluations_per_iteration']))
            else:
                efwa_avg_evals_per_iter.append(0)
        
        # Create a grouped bar chart for average evaluations per iteration
        x = np.arange(len(n_clusters_list))
        width = 0.25
        
        plt.bar(x - width, abc_avg_evals_per_iter, width, label='ABC', color='blue')
        plt.bar(x, aco_avg_evals_per_iter, width, label='ACO', color='red')
        plt.bar(x + width, efwa_avg_evals_per_iter, width, label='EFWA', color='green')
        
        plt.title('Average Evaluations per Iteration')
        plt.xlabel('Number of Clusters')
        plt.ylabel('Evaluations / Iteration')
        plt.xticks(x, n_clusters_list)
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 4. Total execution times
        plt.subplot(2, 2, 4)
        
        # Create a stacked bar chart showing total times
        x = np.arange(len(n_clusters_list))
        width = 0.7
        
        plt.bar(x, results_df['abc_time'], width, label='ABC', color='blue', alpha=0.7)
        plt.bar(x, results_df['aco_time'], width, bottom=results_df['abc_time'], label='ACO', color='red', alpha=0.7)
        plt.bar(x, results_df['efwa_time'], width, bottom=results_df['abc_time']+results_df['aco_time'], label='EFWA', color='green', alpha=0.7)
        
        plt.title('Total Execution Time by Cluster Count')
        plt.xlabel('Number of Clusters')
        plt.ylabel('Time (seconds)')
        plt.xticks(x, n_clusters_list)
        plt.legend(loc='upper left')
        plt.grid(True, alpha=0.3)
        
        plt.suptitle('Efficiency Metrics Across Different Cluster Counts', fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        
        if save_path:
            # Modify the save path to create a separate file
            efficiency_save_path = save_path.replace('.png', '_efficiency_metrics.png')
            plt.savefig(efficiency_save_path)
            print(f"Efficiency metrics visualization saved to {efficiency_save_path}")
        
        plt.close()

    def extract_top_tweets(self, tweets_df_path, embeddings_path, result_dir, n_top=30):
        
        print(f"Extracting top {n_top} tweets per cluster for each algorithm...")
        
        # Load the tweet data
        tweets_df = pd.read_csv(tweets_df_path)
        print(f"Loaded tweets dataframe with shape: {tweets_df.shape}")
        
        # Load embeddings (should be in the same order as tweets_df)
        embeddings = np.load(embeddings_path)
        print(f"Loaded embeddings with shape: {embeddings.shape}")
        
        # Ensure embeddings are normalized
        embeddings = self.data
        
        # Create a directory for top tweets
        top_tweets_dir = os.path.join(result_dir, "top_tweets")
        os.makedirs(top_tweets_dir, exist_ok=True)
        
        # Process each cluster count
        for n_clusters in self.cluster_counts:
            
            # Create a subdirectory for this cluster count
            cluster_dir = os.path.join(top_tweets_dir, f"clusters_{n_clusters}")
            os.makedirs(cluster_dir, exist_ok=True)
            
            # Get results for this cluster count
            if n_clusters not in self.results:
                print(f"No results found for {n_clusters} clusters. Skipping.")
                continue
                
            results = self.results[n_clusters]
            
            # Process each algorithm
            for algo_name, result in results.items():
                if result is None:
                    print(f"No results for {algo_name} with {n_clusters} clusters. Skipping.")
                    continue
                
                # Get assignments and centroids
                assignments = result.get('assignments')
                centroids = result.get('centroids')
                
                if assignments is None or centroids is None:
                    print(f"Missing assignments or centroids for {algo_name}. Skipping.")
                    continue
                    
                # Create algorithm directory
                algo_dir = os.path.join(cluster_dir, algo_name)
                os.makedirs(algo_dir, exist_ok=True)
                
                # Process each cluster
                for cluster in range(n_clusters):
                    # Get indices of tweets assigned to this cluster
                    cluster_indices = np.where(assignments == cluster)[0]
                    
                    # Skip if cluster is empty
                    if len(cluster_indices) == 0:
                        print(f"Cluster {cluster} is empty for {algo_name}. Skipping.")
                        continue
                    
                    # Get embeddings for tweets in this cluster
                    cluster_embeddings = embeddings[cluster_indices]
                    
                    # Get the centroid for this cluster
                    centroid = centroids[cluster].reshape(1, -1)
                    
                    # Compute cosine similarity between each tweet and the centroid
                    similarities = cosine_similarity(cluster_embeddings, centroid).flatten()
                    
                    # Sort indices by similarity (descending)
                    sorted_indices = np.argsort(-similarities)
                    
                    # Get the top N tweets or all if fewer
                    top_count = min(n_top, len(sorted_indices))
                    top_indices = [cluster_indices[i] for i in sorted_indices[:top_count]]
                    
                    # Get the tweets and their similarity scores
                    top_tweets = tweets_df.iloc[top_indices].copy()
                    top_tweets['cosine_similarity_to_centroid'] = similarities[sorted_indices[:top_count]]
                    
                    # Add cluster information
                    top_tweets['cluster'] = cluster
                    top_tweets['algorithm'] = algo_name
                    top_tweets['cluster_size'] = len(cluster_indices)
                    
                    # Save to CSV
                    output_path = os.path.join(algo_dir, f"top_{n_top}_tweets_cluster_{cluster}.csv")
                    top_tweets.to_csv(output_path, index=False)
                    print(f"Saved {top_count} tweets for cluster {cluster} to {output_path}")
                    
                # Create a combined file with all clusters for this algorithm
                all_clusters_df = []
                for cluster in range(n_clusters):
                    cluster_path = os.path.join(algo_dir, f"top_{n_top}_tweets_cluster_{cluster}.csv")
                    if os.path.exists(cluster_path):
                        cluster_df = pd.read_csv(cluster_path)
                        all_clusters_df.append(cluster_df)
                
                if all_clusters_df:
                    combined_df = pd.concat(all_clusters_df, ignore_index=True)
                    combined_path = os.path.join(algo_dir, f"all_clusters_top_{n_top}_tweets.csv")
                    combined_df.to_csv(combined_path, index=False)
                    print(f"Saved combined top tweets for all clusters to {combined_path}")
        
        print("\nCompleted extracting top tweets for all algorithms and cluster counts.")
        return self

In [3]:
# Example usage
data_path = "C:/Users/LENOVO/Documents/00_BU/RESEARCHPAPER/SOM/word2vec_reduced_5wordsOrMore.npy"
tweets_df_path = "C:/Users/LENOVO/Documents/00_BU/RESEARCHPAPER/SOM/tweets_clean_removed.csv"
comparison = AlgorithmComparison(data_path=data_path)

Loading data from C:/Users/LENOVO/Documents/00_BU/RESEARCHPAPER/SOM/word2vec_reduced_5wordsOrMore.npy
Data loaded. Shape: (99988, 300)


In [4]:
comparison.run_comparison(save_results=True, extract_tweets=True, tweets_df_path=tweets_df_path, embeddings_path=data_path)



========== RUNNING COMPARISON WITH 5 CLUSTERS ==========

======= RUNNING ARTIFICIAL BEE COLONY ALGORITHM =======
Saved best initial centroids (score: 1286.44) to initial_centroids_5.npy
ABC Initialization - Fitness: 1286.44, Silhouette: 0.0187


ABC Optimizing:  99% |█████████████████ | ETA:   0:00:08 best_fitness: 2.51e+0333

ABC completed: Fitness = 2509.51, Silhouette = 0.0187, Time = 4152.68s

======= RUNNING ANT COLONY OPTIMIZATION ALGORITHM =======
Centroids loaded from file - same initialization as ABC.-----
ACO Initialization - Fitness: 1184.42, Silhouette: 0.0246


ACO Progress:  99% |███████████████████ | ETA:   0:00:05 best_fitness: 2.61e+0333


Final Results - Best Fitness: 2613.73, Final Silhouette: 0.0051
Total fitness evaluations: 5000
ACO completed: Fitness = 2613.73, Silhouette = 0.0105, Time = 2698.80s

======= RUNNING ENHANCED FIREWORKS ALGORITHM =======
Centroids loaded from file - same initialization as ABC.-----
Unique labels: [0 1 2 3 4]
Current Fitness: 1190.5057537847179


Firework Algorithm:   0%|          | 0/500 [00:00<?, ?it/s]

Cluster counts: [742, 31266, 60867, 381, 6732]
Quality of Centroids: [0.69142956 0.95338768 1.         0.         0.89898008] - Amplitudes: [0.28142783 0.08495924 0.05       0.8        0.12576494] - Spark counts: [11, 2, 2, 20, 3]
Cluster counts: [934, 40139, 47143, 439, 11333]
Quality of Centroids: [0.7047526  1.         0.98440563 0.         0.97243796] - Amplitudes: [0.27143555 0.05       0.06169578 0.8        0.07067153] - Spark counts: [11, 2, 2, 20, 2]
Iteration 1 angular changes (degrees): [0.0, 0.0, 2.7763099273139873, 1.2074182697257333e-06, 0.0]
Cluster counts: [1306, 52977, 24885, 489, 20331]
Quality of Centroids: [0.714409   0.98505826 0.84813934 0.         1.        ] - Amplitudes: [0.26419325 0.0612063  0.1638955  0.8        0.05      ] - Spark counts: [10, 2, 5, 20, 2]
Iteration 2 angular changes (degrees): [0.0, 2.754283695985858, 0.0, 1.2074182697257333e-06, 0.0]
Cluster counts: [1899, 23599, 41534, 683, 32273]
Quality of Centroids: [0.67482498 0.83279267 0.86080055 0.

In [6]:
comparison.extract_top_tweets(tweets_df_path=tweets_df_path, embeddings_path=data_path, result_dir='C:/Users/LENOVO/Documents/00_BU/RESEARCHPAPER/01_FINAL/SEPARATE/comparison_results/run_20250403-232008', n_top=30)

Extracting top 30 tweets per cluster for each algorithm...
Loaded tweets dataframe with shape: (99988, 4)
Loaded embeddings with shape: (99988, 300)

Processing 5 clusters...
Extracting top tweets for abc with 5 clusters...
Cluster 0 is empty for abc. Skipping.
Saved 30 tweets for cluster 1 to C:/Users/LENOVO/Documents/00_BU/RESEARCHPAPER/01_FINAL/SEPARATE/comparison_results/run_20250403-232008\top_tweets\clusters_5\abc\top_30_tweets_cluster_1.csv
Saved 30 tweets for cluster 2 to C:/Users/LENOVO/Documents/00_BU/RESEARCHPAPER/01_FINAL/SEPARATE/comparison_results/run_20250403-232008\top_tweets\clusters_5\abc\top_30_tweets_cluster_2.csv
Saved 30 tweets for cluster 3 to C:/Users/LENOVO/Documents/00_BU/RESEARCHPAPER/01_FINAL/SEPARATE/comparison_results/run_20250403-232008\top_tweets\clusters_5\abc\top_30_tweets_cluster_3.csv
Saved 30 tweets for cluster 4 to C:/Users/LENOVO/Documents/00_BU/RESEARCHPAPER/01_FINAL/SEPARATE/comparison_results/run_20250403-232008\top_tweets\clusters_5\abc\top_30